In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install prog-models

In [2]:
from prog_models.models import CentrifugalPump
import numpy as np
import pandas as pd
import random
from IPython.display import clear_output

In [ ]:
class Pump():

    def __init__(self):
        self.pump = CentrifugalPump(process_noise= 0.000000000001)
        self.pump.parameters['x0']['wA'] = 0.01  # Set Wear Rate
        self.states = self.pump.parameters['x0']
        self.initial_params = self.pump.parameters
        self.V = 471.2389
        self.cycletime= 3600
        self.options = {
                        'save_freq': 100, # Frequency at which results are saved
                        'dt': 2 # Timestep
                        }
        self.temp = 290

    def set_seed(self, s=42):
        np.random.seed(s)
        random.seed(s)

    def future_loading(self,t, x=None):
        return self.pump.InputContainer({
            'Tamb': self.temp,
            'V': self.V,
            'pdisch': 928654,
            'psuc': 239179,
            'wsync': self.V * 0.8
        })

    def get_health(self, action):
        self.V = action['V']
        self.set_temp(action['temp'])
        self.pump.parameters['x0'] = self.states
        (_, _, states, outputs, event_states) = self.pump.simulate_to(action['duration'], self.future_loading, **self.options)
        self.states = states[-1]
        health = event_states[-1]
        # health = self.get_min_health(health)
        return health

    def repair(self):
        # self.pump.parameters =  self.initial_params
        self.pump = CentrifugalPump(process_noise= 0.000000000001)
        self.pump.parameters['x0']['wA'] = 0.01  # Set Wear Rate
        self.states = self.pump.parameters['x0']

    def get_min_health(self, health):
        h = 1
        for k in health.keys():
            if health[k]<h:
                h = health[k]
        return h

    def set_temp(self, celsius_wert):
        result = float(celsius_wert) + 273.15
        self.temp = result


In [4]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [5]:

df = pd.read_pickle('/content/drive/MyDrive/Pump_degradation/pump_degradation.plk')

In [ ]:
df = pd.read_pickle('/content/drive/MyDrive/Pump_degradation/final/results.plk')

In [8]:
df.head()

,w,Q,Tt,Tr,To,A,rThrust,rRadial,wA,wThrust,...,duration,ImpellerWearFailure,ThrustBearingOverheat,RadialBearingOverheat,PumpOilOverheat,ImpellerWearFailure_after,ThrustBearingOverheat_after,RadialBearingOverheat_after,PumpOilOverheat_after,breaks_after_action
0,376.991118,0.000000,290.000000,290.000000,290.000000,12.708400,0.000001,0.000002,0.01,0.000000e+00,...,2520,1.000000,1.000000,1.000000,1.000000,0.906316,0.684101,0.709982,0.652803,0
1,174.411584,-0.055861,315.271959,313.201469,310.831825,12.407825,0.000001,0.000002,0.01,6.575300e-11,...,2520,0.906316,0.684101,0.709982,0.652803,0.986292,0.992862,0.989383,0.984322,0
2,174.411584,-0.055962,315.662622,313.804471,311.445886,12.367965,0.000001,0.000002,0.01,1.022807e-10,...,12060,0.986292,0.992862,0.989383,0.984322,0.891135,0.771285,0.769860,0.671001,0
3,174.411586,-0.056743,328.090408,326.737291,324.130133,12.055743,0.000001,0.000002,0.01,7.104586e-11,...,5040,0.891135,0.771285,0.769860,0.671001,0.850887,1.088980,1.057373,1.112297,0
4,174.411590,-0.057678,324.361281,324.255199,321.225016,11.674649,0.000001,0.000002,0.01,1.243750e-10,...,6300,0.850887,1.088980,1.057373,1.112297,0.896115,0.797745,0.807321,0.708454,0


In [ ]:
# df = pd.DataFrame()

In [ ]:
def make_df(list_of_states, list_of_actions, list_of_healths_before, list_of_healths_after):
    df_states = pd.json_normalize(list_of_states,max_level=1)
    df_actions = pd.json_normalize(list_of_actions,max_level=1)
    df_health_before = pd.json_normalize(list_of_healths_before,max_level=1)
    df_health_after = pd.json_normalize(list_of_healths_after,max_level=1)
    rename_after_cols = {'ImpellerWearFailure': 'ImpellerWearFailure_after',
                     'ThrustBearingOverheat': 'ThrustBearingOverheat_after',
                     'RadialBearingOverheat': 'RadialBearingOverheat_after',
                     'PumpOilOverheat': 'PumpOilOverheat_after'}

    df_health_after.rename(columns=rename_after_cols,inplace=True)

    df_health_after['breaks_after_action'] = df_health_after.min(axis='columns').apply(lambda x: 0 if x>=0 else 1)
    df = pd.concat([df_states,df_actions,df_health_before, df_health_after], axis=1)

    return df

In [ ]:
p = Pump()
p.pump.event_state(p.states)

/usr/local/lib/python3.10/dist-packages/prog_models/models/centrifugal_pump.py:281: UserWarning: wA, wRadial, and wThrust will be overwritten within the model, since the wear rates are part of the state. Use CentrifugalPumpBase to remove this behavior.
  warnings.warn("wA, wRadial, and wThrust will be overwritten within the model, since the wear rates are part of the state. Use CentrifugalPumpBase to remove this behavior.")


{'ImpellerWearFailure': 1.0,
 'ThrustBearingOverheat': 1.0,
 'RadialBearingOverheat': 1.0,
 'PumpOilOverheat': 1.0}

In [ ]:
# p = Pump()


# action =  {'temp': 30, 'V': 230,'duration':1620}

# h = p.get_health(action)
# health =  p.get_min_health(h)
# i = 0
# while health>0:
#   h = p.get_health(action)
#   i +=1
#   print(i)
#   print(h)

#   health =  p.get_min_health(h)


In [ ]:
p = Pump()

product_types = {1: {'temp': 50, 'V': 230},
                 2: {'temp': 45, 'V': 230},
                 3: {'temp': 40, 'V': 230},
                 4: {'temp': 38, 'V': 230},
                 5: {'temp': 33, 'V': 230},
                 6: {'temp': 30, 'V': 230},}



list_of_states = []
list_of_actions = []
list_of_healths_before = []
list_of_healths_after = []

h = p.pump.event_state(p.states)
for i in range(1,10000):
    #jetziger state in die liste
    list_of_states.append(p.states)
    # ranodm itemgruppe
    prod_type_ix = random.randint(1,6)
    action = product_types[prod_type_ix]
    # random zeitspanne
    action['duration'] = random.randint(7,70)*180
    # actions werden abgespeichert
    list_of_actions.append(action)
    # event states bevor der aktio verden gespeichert
    list_of_healths_before.append(h)
    # action wird durchgeführt
    h = p.get_health(action)
    # event states nachd er aktion werden gespeichert
    list_of_healths_after.append(h)
    if p.get_min_health(h)<=0:
      # wenn pumpe kaputt dann werden state und event states auf as good as new gesetzt
      p.repair()
      h = p.pump.event_state(p.states)

    if i%300==0:
      df_upload = make_df(list_of_states, list_of_actions, list_of_healths_before, list_of_healths_after)
      df_upload = pd.concat([df,df_upload])
      df_upload.to_pickle('/content/drive/MyDrive/Pump_degradation/pump_degradation.plk')
      clear_output(wait=True)
      print(i)


4800


/usr/local/lib/python3.10/dist-packages/prog_models/models/centrifugal_pump.py:281: UserWarning: wA, wRadial, and wThrust will be overwritten within the model, since the wear rates are part of the state. Use CentrifugalPumpBase to remove this behavior.
  warnings.warn("wA, wRadial, and wThrust will be overwritten within the model, since the wear rates are part of the state. Use CentrifugalPumpBase to remove this behavior.")
/usr/local/lib/python3.10/dist-packages/prog_models/models/centrifugal_pump.py:281: UserWarning: wA, wRadial, and wThrust will be overwritten within the model, since the wear rates are part of the state. Use CentrifugalPumpBase to remove this behavior.
  warnings.warn("wA, wRadial, and wThrust will be overwritten within the model, since the wear rates are part of the state. Use CentrifugalPumpBase to remove this behavior.")
/usr/local/lib/python3.10/dist-packages/prog_models/models/centrifugal_pump.py:281: UserWarning: wA, wRadial, and wThrust will be overwritten wi

KeyboardInterrupt: ignored

In [ ]:
# df.loc[pd.isnull(df['breaks_after_action'])]
df_upload

In [ ]:
# df.drop(ix)

In [ ]:
# df_states = pd.json_normalize(list_of_states,max_level=1)
# df_actions = pd.json_normalize(list_of_actions,max_level=1)
# df_health_before = pd.json_normalize(list_of_healths_before,max_level=1)
# df_health_after = pd.json_normalize(list_of_healths_after,max_level=1)



In [ ]:
# rename_after_cols = {'ImpellerWearFailure': 'ImpellerWearFailure_after',
#                      'ThrustBearingOverheat': 'ThrustBearingOverheat_after',
#                      'RadialBearingOverheat': 'RadialBearingOverheat_after',
#                      'PumpOilOverheat': 'PumpOilOverheat_after'}

# df_health_after.rename(columns=rename_after_cols,inplace=True)

# df_health_after['breaks_after_action'] = df_health_after.min(axis='columns').apply(lambda x: 0 if x>=0 else 1)

In [ ]:
# df_health_after

In [ ]:
# df = pd.concat([df_states,df_actions,df_health_before, df_health_after], axis=1)

In [ ]:
# df[df['breaks_after_action']==1]

In [ ]:
# df.to_pickle('/content/drive/MyDrive/Pump_degradation/pump_degradation.plk')